<a href="https://colab.research.google.com/github/DANU011/practice-analysis/blob/main/coding_practice/2%EC%9C%A0%ED%98%95%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classifier

In [ ]:
# 서비스 이탈예측 데이터
# 데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
'''
x_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv
y_train : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv
x_test : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_test.csv
데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형 ''';

import pandas as pd

# 데이터 로드
X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

display(X_train.head())
display(y_train.head())

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,15799217,Zetticci,791,Germany,Female,35,7,52436.20,1,1,0,161051.75
1,15748986,Bischof,705,Germany,Male,42,8,166685.92,2,1,1,55313.51
2,15722004,Hsiung,543,France,Female,31,4,138317.94,1,0,0,61843.73
3,15780966,Pritchard,709,France,Female,32,2,0.00,2,0,0,109681.29
4,15636731,Ts'ai,714,Germany,Female,36,1,101609.01,2,1,1,447.73


,CustomerId,Exited
0,15799217,0
1,15748986,0
2,15722004,0
3,15780966,0
4,15636731,0


In [ ]:
print(X_train.info())
print(X_train.nunique())
print(X_train.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 609.4+ KB
None
CustomerId         6499
Surname            2289
CreditScore         459
Geography             3
Gender                4
Age                  69
Tenure               11
Balance     

In [ ]:
# 데이터 전처리
drop_columns = ['CustomerId', 'Surname']
X_train_drop = X_train.drop(columns = drop_columns)
X_test_drop = X_test.drop(columns = drop_columns)

In [ ]:
import sklearn
print(sklearn.__all__)
import sklearn.model_selection
print(dir(sklearn.model_selection))

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']
['BaseCrossValidator', 'BaseShuffleSplit', 'GridSearchCV', 'GroupKFold', 'GroupShuffleSplit', 'KFold', 'LearningCurveDisplay', 'LeaveOneGroupOut', 'LeaveOneOut', 'LeavePGroupsOut', 'LeavePOut', 'ParameterGrid', 'ParameterSampler', 'PredefinedSplit', 'RandomizedSearchCV', 'RepeatedKFold', 'RepeatedStratifiedKFold', 'ShuffleSplit', 'StratifiedGroupKFold', 'StratifiedK

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 원핫 인코딩

# 데이터프레임의 범주형 변수를 원핫 인코딩. 각 범주형 변수는 새로운 이진 열로 변환.
X_train_dummies = pd.get_dummies(X_train_drop)
y = y_train['Exited'] # target 변수

# 테스트 데이터에 대해서도 동일한 작업을 수행.
# 훈련 데이터와 동일한 원핫 인코딩 구조를 갖기 위해 훈련 데이터와 같은 열 순서를 갖도록함.
X_test_dummies = pd.get_dummies(X_test_drop)
# 훈련 데이터와 테스트 데이터를 원핫 인코딩한 후에는 두 데이터프레임이 동일한 특성(열)을 가져야 함.
# 테스트 데이터의 열 순서를 훈련 데이터와 동일하게 맞춰주는 과정.
# 만약 테스트 데이터에 훈련 데이터에 없는 새로운 범주가 있으면 해당 열은 버려짐.
X_test_dummies = X_test_dummies[X_train_dummies.columns]

# print(help(train_test_split))
X_train, X_validation, y_train, y_validation = train_test_split(X_train_dummies, y, test_size = 0.33, random_state = 42)
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=23)

In [ ]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# model_score 훈련 및 검증 세트에 대한 예측 및 양성 클래스의 예측 확률을 생성

# 훈련 데이터에 대한 예측을 생성
# rf.predict 메서드는 입력 데이터에 대한 모델의 예측된 클래스(여기서는 이탈 여부)를 반환.
predict_train_label = rf.predict(X_train)

# 훈련 데이터에 대한 양성 클래스(이탈)의 예측 확률을 생성
# rf.predict_proba 메서드는 각 클래스에 대한 확률을 반환 [:, 1]는 양성 클래스의 확률만을 선택.
predict_train_proba = rf.predict_proba(X_train)[:,1]

# 검증 데이터에 대한 예측을 생성
predict_validation_label = rf.predict(X_validation)
# 검증 데이터에 대한 양성 클래스의 예측 확률을 생성
predict_validation_prob = rf.predict_proba(X_validation)[:,1]

In [ ]:
# 문제에서 묻는 것에 따라 모델 성능 확인
# 정확도 (accuracy) , f1_score , recall , precision -> model.predict
# auc , 확률이라는 표현 -> model.predict_proba로 결과 출력 첫번째 행의 값을 가져오기 model.predict_proba()[:,1]

# 모델 지표 출력
print('train accuracy :', accuracy_score(y_train,predict_train_label))
print('validation accuracy :', accuracy_score(y_validation,predict_validation_label))
print('\n')
print('train f1_score :', f1_score(y_train,predict_train_label))
print('validation accuracy :', f1_score(y_validation,predict_validation_label))
print('\n')
print('train recall_score :', recall_score(y_train,predict_train_label))
print('validation recall_score :', recall_score(y_validation,predict_validation_label))
print('\n')
print('train precision_score :', precision_score(y_train,predict_train_label))
print('validation precision_score :', precision_score(y_validation,predict_validation_label))
print('\n')
print('train auc :', roc_auc_score(y_train,predict_train_proba))
print('validation auc :', roc_auc_score(y_validation,predict_validation_prob))

# test 데이터에 대한 모델 예측 마찬가지로 위와 같은 방식
predict_test_label = rf.predict(X_test_dummies)
predict_test_proba = rf.predict_proba(X_test_dummies)[:,1]

# csv로 예측 저장
#
# accuracy, f1_score, recall, precision
#pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_label}).to_csv('003000000.csv', index=False)

# auc, 확률
#pd.DataFrame({'CustomerId': x_test.CustomerId, 'Exited': predict_test_proba}).to_csv('003000000.csv', index=False)

train accuracy : 1.0
validation accuracy : 0.8652680652680653


train f1_score : 1.0
validation accuracy : 0.5912305516265912


train recall_score : 1.0
validation recall_score : 0.4543478260869565


train precision_score : 1.0
validation precision_score : 0.8461538461538461


train auc : 1.0
validation auc : 0.8497613211198555


In [ ]:
# 이직여부 판단 데이터
# 데이터 설명 : 이직여부 판단 데이터 (target: 1: 이직 , 0 : 이직 x)
'''x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists (참고, 데이터 수정)''';

import pandas as pd
X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv')

display(X_train.head())
display(y_train.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17


,enrollee_id,target
0,25298,0.0
1,4241,1.0
2,24086,0.0
3,26773,0.0
4,32325,1.0


In [ ]:
print(X_train.info())
print('\n')
print(X_train.nunique())
print('\n')
print(X_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 1.2+ MB
None


enrollee_id               

In [ ]:
# 범주형 변수인데 많은 unique값을 가진 컬럼은 날림.
drop_columns = ['enrollee_id', 'city', 'city_development_index', 'experience', 'training_hours']
X_train_drop = X_train.drop(columns = drop_columns)
X_test_drop = X_test.drop(columns = drop_columns)

In [ ]:
import sklearn
import sklearn.model_selection
print(sklearn.__all__)
print(dir(sklearn.model_selection))

['calibration', 'cluster', 'covariance', 'cross_decomposition', 'datasets', 'decomposition', 'dummy', 'ensemble', 'exceptions', 'experimental', 'externals', 'feature_extraction', 'feature_selection', 'gaussian_process', 'inspection', 'isotonic', 'kernel_approximation', 'kernel_ridge', 'linear_model', 'manifold', 'metrics', 'mixture', 'model_selection', 'multiclass', 'multioutput', 'naive_bayes', 'neighbors', 'neural_network', 'pipeline', 'preprocessing', 'random_projection', 'semi_supervised', 'svm', 'tree', 'discriminant_analysis', 'impute', 'compose', 'clone', 'get_config', 'set_config', 'config_context', 'show_versions']
['BaseCrossValidator', 'BaseShuffleSplit', 'GridSearchCV', 'GroupKFold', 'GroupShuffleSplit', 'KFold', 'LearningCurveDisplay', 'LeaveOneGroupOut', 'LeaveOneOut', 'LeavePGroupsOut', 'LeavePOut', 'ParameterGrid', 'ParameterSampler', 'PredefinedSplit', 'RandomizedSearchCV', 'RepeatedKFold', 'RepeatedStratifiedKFold', 'ShuffleSplit', 'StratifiedGroupKFold', 'StratifiedK

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train_dummies = pd.get_dummies(X_train_drop)
y = y_train['target'].astype('int')

X_test_dummies = pd.get_dummies(X_test_drop)
# train과 컬럼 순서 맞추기
X_test_dummies = X_test_dummies[X_train_dummies.columns]

# RandomForest 모델 훈련
X_train, X_validation, y_train, y_validation = train_test_split(X_train_dummies, y, test_size = 0.33, random_state = 42)
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=23)

In [ ]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# model_score
predict_train_label = rf.predict(X_train)
predcit_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_proba = rf.predict_proba(X_validation)[:,1]

# 모델 성능 확인
print('train acc :', accuracy_score(y_train, predict_train_label))
print('val acc :', accuracy_score(y_validation, predict_validation_label))
print('\n')
print('train f1 :', f1_score(y_train, predict_train_label))
print('val f1 :', f1_score(y_validation, predict_validation_label))
print('\n')
print('train pre :', precision_score(y_train, predict_train_label))
print('val pre :', precision_score(y_validation, predict_validation_label))
print('\n')
print('train rec :', recall_score(y_train, predict_train_label))
print('val rec :', recall_score(y_validation, predict_validation_label))
print('\n')
print('train roc :', roc_auc_score(y_train, predcit_train_proba))
print('val roc :', roc_auc_score(y_validation, predict_validation_proba))

train acc : 0.8334931671062096
val acc : 0.7255474452554744


train f1 : 0.5862377122430742
val f1 : 0.32777115613825986


train pre : 0.7610208816705336
val pre : 0.4317111459968603


train rec : 0.47674418604651164
val rec : 0.2641690682036503


train roc : 0.8762173756146109
val roc : 0.6376763826796363


In [ ]:
# test data
predict_test_label = rf.predict(X_test_dummies)
predict_test_proba = rf.predict_proba(X_test_dummies)[:,1]

# accuracy, f1_score, recall, precision
pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': predict_test_label}).to_csv('003000000.csv', index=False)

# auc, 확률
pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': predict_test_proba}).to_csv('003000000.csv', index=False)

In [ ]:
# 정시 배송 여부 판단
#데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

''' x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/prachi13/customer-analytics (참고, 데이터 수정)''';

import pandas as pd
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')

display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [ ]:
print(x_train.info())
print(x_train.nunique())
print(x_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB
None
ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Pro

In [ ]:
# 범주형 변수인데 많은 unique값을 가진 컬럼 삭제.
drop_columns = ['ID']
x_train_drop = x_train.drop(columns = drop_columns)
x_test_drop = x_test.drop(columns = drop_columns)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 더미화
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['Reached.on.Time_Y.N']

x_test_dummies = pd.get_dummies(x_test_drop)
# 컬럼 순서 맞추기
x_test_dimmies = x_test_dummies[x_train_dummies.columns]

# 데이터 분할
x_train, x_validation, y_train, y_validation = train_test_split(x_train_dummies, y, test_size = 0.33, random_state = 42)

# 모델 학습
rf = RandomForestClassifier(random_state = 23)
rf.fit(x_train, y_train)

RandomForestClassifier(random_state=23)

In [ ]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

# model_score
predict_train_label = rf.predict(x_train)
predict_train_proba = rf.predict_proba(x_train)[:, 1]

predict_validation_label = rf.predict(x_validation)
predict_validation_proba = rf.predict_proba(x_validation)[:, 1]

# 결과 확인
print('train accuracy_score :', accuracy_score(y_train, predict_train_label))
print('val accuracy_score :', accuracy_score(y_validation, predict_validation_label))
print('\n')
print('train f1_score :', f1_score(y_train, predict_train_label))
print('val f1_score :', f1_score(y_validation, predict_validation_label))
print('\n')
print('train precision_score :', precision_score(y_train, predict_train_label))
print('val precision_score :', precision_score(y_validation, predict_validation_label))
print('\n')
print('train recall_score :', recall_score(y_train, predict_train_label))
print('val recall_score :', recall_score(y_validation, predict_validation_label))
print('\n')
print('train roc_auc_score :', roc_auc_score(y_train, predict_train_proba))
print('val roc_auc_score :', roc_auc_score(y_validation, predict_validation_proba))

train accuracy_score : 1.0
val accuracy_score : 0.6395775941230487


train f1_score : 1.0
val f1_score : 0.6744089589382


train precision_score : 1.0
val precision_score : 0.7245989304812834


train recall_score : 1.0
val recall_score : 0.630721489526765


train roc_auc_score : 1.0
val roc_auc_score : 0.7261997118475008


In [ ]:
# test data
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]

In [ ]:
# 비행탑승 경험 만족도 데이터
# 데이터 설명 : 비행탑승 경험 만족도 (satisfaction 컬럼 : ‘neutral or dissatisfied’ or satisfied ) (83123, 24) shape
''' x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_test.csv
데이터 출처 :https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction?select=train.csv (참고, 데이터 수정)''';

import pandas as pd


## Regressor

In [2]:
# 학생성적 예측 데이터
# 데이터 설명 : 학생성적 예측 (종속변수 :G3)
''' x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/ishandutta/student-performance-data-set (참고, 데이터 수정)''';

import pandas as pd

x_train = pd.read_csv(' https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/studentscore/y_train.csv')

display(x_train.head())
display(y_train.head())

,StudentID,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,1714,GP,F,18,U,GT3,T,4,3,other,...,no,4,3,3,1,1,3,0,14,13
1,1254,GP,F,17,U,GT3,T,4,3,health,...,yes,4,4,3,1,3,4,0,13,15
2,1639,GP,F,16,R,GT3,T,4,4,health,...,no,2,4,4,2,3,4,6,10,11
3,1118,GP,M,16,U,GT3,T,4,4,services,...,no,5,3,3,1,3,5,0,15,13
4,1499,GP,M,19,U,GT3,T,3,2,services,...,yes,4,5,4,1,1,4,0,5,0


,StudentID,G3
0,1714,14
1,1254,15
2,1639,11
3,1118,13
4,1499,0


In [3]:
x_train.isnull().sum()
x_test.isnull().sum()

StudentID     0
school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import numpy as np

drop_columns = ['StudentID']

x_train_drop = x_train.drop(columns = drop_columns)
x_test_drop = x_test.drop(columns = drop_columns)
y_train_target = y_train['G3']

x_train_dummies = pd.get_dummies(x_train_drop)
x_test_dummies = pd.get_dummies(x_test_drop)[x_train_dummies.columns]

xtr, xt, ytr, yt = train_test_split(x_train_dummies, y_train_target)

rf = RandomForestRegressor()
rf.fit(xtr, ytr)

y_validation_pred = rf.predict(xt)

# 모델평가
# mse : mean_squared_error / mae : mean_absolute_error  / mape : mean_absolute_percentage_error
# rmse : root_mean_squerd_error -> 패키지 없음 np.sqrt(mean_squared_error)

# y_true = 실제값 ,y_pred = 예측 순서 잘 확인!

# print(help(mean_squared_error))

#mse
print('validation mse' ,mean_squared_error(yt,y_validation_pred))

#mae
print('validation mae' ,mean_absolute_error(yt,y_validation_pred))

#mape
print('validation mape' ,mean_absolute_percentage_error(yt,y_validation_pred))

#rmse
print('validation rmse' ,np.sqrt(mean_absolute_percentage_error(yt,y_validation_pred)))

#r2
print('validation r2 score' ,r2_score(yt,y_validation_pred))

validation mse 2.6959839088235293
validation mae 0.9888882352941176
validation mape 477116642876133.2
validation rmse 21842999.85982084
validation r2 score 0.8007846123665316


In [33]:
# 중고차 가격 예측 데이터
# 데이터 설명 : 중고차 가격 예측 데이터 (종속변수 :price)
'''x_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv
y_train: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv
x_test: https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv
x_label(평가용) : https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_test.csv
데이터 출처 :https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes?select=vw.csv (참고, 데이터 수정)''';

import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv')

display(x_train.head())
display(x_test.head())
display(y_train.head())
print(x_train.nunique())
print(x_test.nunique())

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,12000,merc,GLS Class,2017,Automatic,12046,Diesel,150.0,37.2,3.0
1,12001,vw,Amarok,2017,Automatic,37683,Diesel,260.0,36.2,3.0
2,12004,merc,GLS Class,2019,Automatic,10000,Diesel,145.0,34.0,3.0
3,12013,skoda,Scala,2019,Manual,3257,Petrol,145.0,49.6,1.0
4,12017,audi,RS6,2015,Semi-Auto,20982,Petrol,325.0,29.4,4.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


carID           4960
brand              9
model             90
year              24
transmission       4
mileage         3900
fuelType           5
tax               41
mpg              125
engineSize        37
dtype: int64
carID           2672
brand              9
model             89
year              23
transmission       4
mileage         2226
fuelType           5
tax               34
mpg              107
engineSize        32
dtype: int64


In [34]:
# train에만 있고 test에는 없는 model들이 있기에 합쳐서 원핫 인코딩을 진행
# data leakage 이슈가 있으나 빅분기 시험에서는 이러한 방식으로 해결하는 것으로 태클걸지 않음
# 정석은 train과 동일한 컬럼을 만들고 test에서 없는 경우에는 0으로 채우고, test에는 있으나 train에 없는 경우에는 데이터 제거
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

drop_columns = ['carID']

x_train_drop = x_train.drop(columns = drop_columns)
x_test_drop = x_test.drop(columns = drop_columns)[x_train_drop.columns]
y_train_target = y_train['price']

combined = pd.concat([x_train_drop, x_test_drop])
combined_encoded = pd.get_dummies(combined)
train_encoded = combined_encoded[:len(x_train_drop)]
test_encoded = combined_encoded[len(x_train_drop):]


xtr, xt, ytr, yt = train_test_split(train_encoded, y_train_target, test_size = 0.33, random_state = 42)

rf = RandomForestRegressor(random_state = 23)
rf.fit(xtr, ytr)

y_validation_pred = rf.predict(xt)

In [35]:
import sklearn.metrics
print(dir(sklearn.metrics))

['ConfusionMatrixDisplay', 'DetCurveDisplay', 'DistanceMetric', 'PrecisionRecallDisplay', 'PredictionErrorDisplay', 'RocCurveDisplay', 'SCORERS', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_base', '_classification', '_dist_metrics', '_pairwise_distances_reduction', '_pairwise_fast', '_plot', '_ranking', '_regression', '_scorer', 'accuracy_score', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'auc', 'average_precision_score', 'balanced_accuracy_score', 'brier_score_loss', 'calinski_harabasz_score', 'check_scoring', 'class_likelihood_ratios', 'classification_report', 'cluster', 'cohen_kappa_score', 'completeness_score', 'confusion_matrix', 'consensus_score', 'coverage_error', 'd2_absolute_error_score', 'd2_pinball_score', 'd2_tweedie_score', 'davies_bouldin_score', 'dcg_score', 'det_curve', 'euclidean_distances', 'explained_variance_score', 'f1_score', 'fbeta_score', 'fowlkes_mallows_score', 'ge

In [36]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import numpy as np

# 모델 평가
# mean_absolute_error
print('mae :', mean_absolute_error(yt, y_validation_pred))
# mean_absolute_percentage_error
print('mape :', mean_absolute_percentage_error(yt, y_validation_pred))
# mean_squared_error
print('mse :', mean_squared_error(yt, y_validation_pred))
# root_mean_squared_error
print('rmse :', np.sqrt(mean_squared_error(yt, y_validation_pred)))
# r2_score
print('r2 :', r2_score(yt, y_validation_pred))

mae : 2205.895988978373
mape : 0.10665481251864602
mse : 17529155.091796063
rmse : 4186.783382478256
r2 : 0.9363581815226119


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
import numpy as np

# 데이터 불러오기
x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/X_test.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/carsprice/y_train.csv')

# 데이터 확인
display(x_train.head())
display(x_test.head())
display(y_train.head())

# train 데이터의 'model' 컬럼이 test 데이터에는 없는 경우가 있으므로 둘을 합친 후 원핫 인코딩을 진행
combined = pd.concat([x_train, x_test])

# 'model'을 기준으로 원핫 인코딩
combined_encoded = pd.get_dummies(combined)

# train과 test로 다시 나누기
train_encoded = combined_encoded[:len(x_train)]
test_encoded = combined_encoded[len(x_train):]

# train 데이터의 target 컬럼 선택
y_train_target = y_train['price']

# train 데이터를 train set과 validation set으로 나누기
xtr, xt, ytr, yt = train_test_split(train_encoded, y_train_target, test_size = 0.33, random_state = 42)

# 랜덤 포레스트 모델 생성 및 학습
rf = RandomForestRegressor(random_state = 23)
rf.fit(xtr, ytr)

# validation set에 대한 예측
y_validation_pred = rf.predict(xt)

# 모델 평가
# mean_absolute_error
print('mae:', mean_absolute_error(yt, y_validation_pred))
# mean_absolute_percentage_error
print('mape:', mean_absolute_percentage_error(yt, y_validation_pred))
# mean_squared_error
print('mse:', mean_squared_error(yt, y_validation_pred))
# root_mean_squared_error
print('rmse:', np.sqrt(mean_squared_error(yt, y_validation_pred)))
# r2_score
print('r2:', r2_score(yt, y_validation_pred))


,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,12000,merc,GLS Class,2017,Automatic,12046,Diesel,150.0,37.2,3.0
1,12001,vw,Amarok,2017,Automatic,37683,Diesel,260.0,36.2,3.0
2,12004,merc,GLS Class,2019,Automatic,10000,Diesel,145.0,34.0,3.0
3,12013,skoda,Scala,2019,Manual,3257,Petrol,145.0,49.6,1.0
4,12017,audi,RS6,2015,Semi-Auto,20982,Petrol,325.0,29.4,4.0


,carID,price
0,13207,31995
1,17314,7700
2,12342,58990
3,13426,12999
4,16004,16990


mae: 2244.325565058033
mape: 0.10835036745136183
mse: 17267097.098027796
rmse: 4155.369670441824
r2: 0.9373096162713269
